<a href="https://colab.research.google.com/github/jackdaus/egolifter/blob/colab/01_colab-demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Get Data Access Link

Before you can download ADT data, you need to request from Aria the `ADT_download_urls.json`. This is an insant process. Follow the [directions in the original egolifter github](https://github.com/facebookresearch/egolifter?tab=readme-ov-file#download-and-pre-processing).

In [ ]:
# IMPORTANT: Make sure you have uploaded your file `ADT_download_urls.json`!
# See original repo for instructions on how to get that.
import os

file_path = "/content/ADT_download_urls.json"

if os.path.exists(file_path):
  print("File found!", file_path)
else:
  print("Error: File ADT_download_urls.json not found!", file_path)
  print("Please upload the ADT_download_urls.json file to the /content directory.")
  print("See the original EgoLifter repository for how to get this download link.")
  raise FileNotFoundError("ADT_download_urls.json not found")

# Install Dependencies

In [ ]:
# Install uv package manager
!pip install uv

# Verify install
!uv --version

In [ ]:
# Clone the repo
!git clone https://github.com/jackdaus/egolifter.git
%cd egolifter

In [ ]:
# Install packages. This might take a few minutes.
# NOTE: we have a hacky workaround here for Colab. Colab doesn't seem
# to want to install groundingdino. So we remove it, then install other deps,
# and then install groundingdino without isolation. (There is probably a better
# way to do this...)
!uv remove groundingdino
!uv sync

In [ ]:
# Then install groundingdino (without build isolation). This is to ensure that it uses pytorch=2.5.1
!uv add "groundingdino @ git+https://github.com/IDEA-Research/Grounded-Segment-Anything#subdirectory=GroundingDINO" --no-build-isolation-package groundingdino

In [ ]:
# Note that in order to use the uv virtual environment, we must activate it within
# the cell. (Colab was giving me trouble creating a uv based kernel...)

# To illustrate, notice that the output of the two commands shows different locations
# of the python exectuable in use.
!source .venv/bin/activate; which python

!which python

In [ ]:
# Unfortunately, we must do something convoluted. We use Jupyter notebook magic
# to create a bash script. We then activate the venv. We then run some python code.
# We should see PyTorch version 2.5.1.
%%bash
source .venv/bin/activate

python - <<'PY'
import torch
import torchvision
print("PyTorch version:", torch.__version__)
print("Torchvision version:", torchvision.__version__)
print("CUDA is available:", torch.cuda.is_available())
PY

In [ ]:
# Set environment variables (adpated from setup_env.bash in original egolifter)
import os
os.environ.update({
    "EGOLIFTER_PATH":                 "/content/egolifter",
    "GSA_PATH":                       "/content/egolifter/Grounded-Segment-Anything",
    "SAM_CHECKPOINT_PATH":            "/content/egolifter/sam_vit_h_4b8939.pth",
    "GROUNDING_DINO_CHECKPOINT_PATH": "/content/egolifter/groundingdino_swint_ogc.pth",
    "SAM_ENCODER_VERSION":            "vit_h",
    "GROUNDING_DINO_CONFIG_PATH":     "/content/egolifter/Grounded-Segment-Anything/GroundingDINO/groundingdino/config/GroundingDINO_SwinT_OGC.py",
    "EFFICIENTSAM_PATH":              "/content/egolifter/Grounded-Segment-Anything/EfficientSAM",
    "TAG2TEXT_PATH":                  "/content/egolifter/Grounded-Segment-Anything/Tag2Text",
    "ADT_DATA_ROOT":                  "adt",
    "ADT_PROCESSED_ROOT":             "adt_processed",
    "AM_I_DOCKER":                    "False",
    "BUILD_WITH_CUDA":                "True",
    "TAG2TEXT_CHECKPOINT_PATH":       "/content/egolifter/Grounded-Segment-Anything/Tag2Text/tag2text_swin_14m.pth",
    "RAM_CHECKPOINT_PATH":            "/content/egolifter/Grounded-Segment-Anything/Tag2Text/ram_swin_large_14m.pth",
})

In [ ]:
# Download model weights
!wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth
!wget https://github.com/IDEA-Research/GroundingDINO/releases/download/v0.1.0-alpha/groundingdino_swint_ogc.pth

# Download Data

Make sure you've uploaded your `ADT_download_urls.json` file. Then run the code below.

## Prepare to download data

In [ ]:
%%bash
source .venv/bin/activate

python - <<'PY'

import os
import shutil

# Define data directories
ADT_DATA_ROOT = os.environ['ADT_DATA_ROOT']
ADT_PROCESSED_ROOT = os.environ['ADT_PROCESSED_ROOT']

# Create directories if they don't exist
os.makedirs(ADT_DATA_ROOT, exist_ok=True)
os.makedirs(ADT_PROCESSED_ROOT, exist_ok=True)

# Copy the download URLs JSON file
source_path = "/content/ADT_download_urls.json"  # Update if needed
destination_path = os.path.join(ADT_DATA_ROOT, "ADT_download_urls.json")
shutil.copy(source_path, destination_path)

PY

In [ ]:
# Copy the vignette images to the dataset
!cp assets/vignette_imx577.png ${ADT_DATA_ROOT} # Vignette image for the RGB camera
!cp assets/vignette_ov7251.png ${ADT_DATA_ROOT} # Vignette image for the SLAM camera

In [ ]:
# Move the vignette files
import shutil

# Define the destination paths within the data directory
vignette_rgb_destination = os.path.join(os.environ['ADT_DATA_ROOT'], "vignette_imx577.png")
vignette_slam_destination = os.path.join(os.environ['ADT_DATA_ROOT'], "vignette_ov7251.png")

# Copy the vignette images
shutil.copy("assets/vignette_imx577.png", vignette_rgb_destination)
shutil.copy("assets/vignette_ov7251.png", vignette_slam_destination)

## Actually download data

In [ ]:
# Define the scene names in a Python list. For now, we just have one sample scene.
scene_names_new = ["Apartment_release_golden_skeleton_seq100_10s_sample_M1292"]
scene_names     = ["Apartment_release_golden_skeleton_seq100_10s_sample"]

# Loop through the scene names and execute the download command for each
for scene_name in scene_names_new:
    !uvx --from projectaria-tools aria_dataset_downloader \
        -c adt/ADT_download_urls.json \
        -o adt/ \
        -d 0 1 2 3 6 7 \
        -l {scene_name}

# Process Data (Part 1)

In [ ]:
# Process the images to be in a format appropriate for 3dgs
for scene_name in scene_names:
  !uv run python scripts/process_adt_3dgs.py \
    --data_root adt \
    --output_root adt_processed \
    --sequence_name {scene_name}

# Train on Vanilla 3DGS Pipeline

This is a first test of training on the vanilla 3DGS pipeline.

In [ ]:
# Run the vanilla 3dgs pipeline.
# It will ask if you want to log into wandb to visualize training progress/logs
# Run the code. In Colab, we must limit number of worker threads to 2.
# Set this to True to run the sample. We won't this by default. But this can
# be a good first test before moving on to the more complicated stuff below.
run_vanilla_3dgs_sample = True

if  run_vanilla_3dgs_sample:
  !uv run python train_lightning.py \
    scene.scene_name=Apartment_release_golden_skeleton_seq100_10s_sample \
    scene.data_root=$ADT_PROCESSED_ROOT \
    exp_name=3dgs \
    output_root=./output/adt \
    wandb.project=egolifter_adt \
    scene.num_workers=2

# Process Data (Part 2)

## Segmentation

In [ ]:
# Generate the SAM segmentation results. This takes about 20 to 30 minutes on a T4,
# or about 10 minutes with an A100.
!uv run python scripts/generate_gsa_results.py \
  -i adt_processed/Apartment_release_golden_skeleton_seq100_10s_sample \
  --class_set none \
  --sam_variant sam \
  --max_longer_side 512 \
  --no_clip

## Generate evaluation target for query-based segmentation

### generate_2dseg_query

In [ ]:
# This takes about 4 minutes
%%bash
source .venv/bin/activate

SCENE_NAME="Apartment_release_golden_skeleton_seq100_10s_sample"

uv run python scripts/generate_2dseg_query.py \
  --data_root $ADT_PROCESSED_ROOT \
  --scene_name $SCENE_NAME

### generate_2dseg_query_sample

In [ ]:
%%bash
source .venv/bin/activate

SCENE_NAME="Apartment_release_golden_skeleton_seq100_10s_sample"

uv run python scripts/generate_2dseg_query_sample.py \
  --data_root $ADT_PROCESSED_ROOT \
  --scene_name $SCENE_NAME

### generate_3dbox_query

In [ ]:
%%bash
source .venv/bin/activate

SCENE_NAME="Apartment_release_golden_skeleton_seq100_10s_sample"

uv run python scripts/generate_3dbox_query.py \
  --raw_root $ADT_DATA_ROOT \
  --data_root $ADT_PROCESSED_ROOT \
  --scene_name $SCENE_NAME

# EgoLifter (full method)

In [ ]:
# EgoLifter (full method). This takes about 2 hours on an A100 or
# about 5 hours on T5.
# Note that on Colab, we must limit the number of worker threads to 2.
!uv run python train_lightning.py \
    scene.scene_name="Apartment_release_golden_skeleton_seq100_10s_sample" \
    scene.data_root=$ADT_PROCESSED_ROOT \
    model=unc_2d_unet \
    model.unet_acti=sigmoid \
    model.dim_extra=16 \
    lift.use_contr=True \
    exp_name=egolifter \
    output_root=./output/adt \
    wandb.project=egolifter_adt \
    scene.num_workers=2

# Save results
We can save our results to Google Drive to persist them.

In [ ]:
import shutil, pathlib
from google.colab import drive

save_results = True

if save_results:
  drive.mount('/content/drive')

  # You can change these paths, as needed
  src = pathlib.Path('/content/egolifter/output')
  dst = pathlib.Path('/content/drive/MyDrive/egolifter/output')
  shutil.copytree(src, dst, dirs_exist_ok=True)

In [ ]:
# Also, we can optionally save the preprocessed data. This will speed things
# up in the future - we don't need to re-preprocess the data!
save_processed_data = True

if save_processed_data:
  # You can change these paths, as needed
  src = pathlib.Path('/content/egolifter/adt_processed')
  dst = pathlib.Path('/content/drive/MyDrive/egolifter/')
  shutil.copytree(src, dst, dirs_exist_ok=True)